In [ ]:
import pandas as pd
import pickle
import numpy as np
from brainflow.board_shim import BoardShim, BrainFlowInputParams, LogLevels, BoardIds
from brainflow.data_filter import DataFilter, FilterTypes
import matplotlib
import os
import torch
import pytorch_lightning as pl
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from pytorch_lightning.loggers import WandbLogger
from torchmetrics.classification import BinaryF1Score, BinaryAccuracy
from pytorch_lightning.callbacks import Callback
from sklearn import metrics

import cv2
from torch.utils.data import DataLoader
import wandb

matplotlib.use('Agg')
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
%matplotlib inline
scaler = StandardScaler()

In [ ]:
wandb.login()
api_key = 'a45abb01f9556b57620ce77c8984452bee7a8772'
board_id = 38
sf = 256

In [ ]:
class PickleLatentDatasetLoader(Dataset):
    def __init__(self, annotations_file, dir):
        self.file_lists = pd.read_csv(annotations_file, header=None)
        self.dir = dir

    def __len__(self):
        return len(self.file_lists)

    def __getitem__(self, idx):
        file_path = os.path.join(self.dir, self.file_lists.iloc[idx, 0].replace('/','\\'))
        label = self.file_lists.iloc[idx, 1]

        pkl_file = open(file_path, 'rb')
        results = pickle.load(pkl_file)
        results = np.array(results)
        pkl_file.close()

        results = torch.tensor(results)
        label = torch.tensor(label).type(torch.FloatTensor)

        return (results[0].squeeze(), results[1].squeeze(), label)

In [ ]:
pkl_dir = 'D:\\Nicko\\TUGAS_AKHIR\\Dataset_new_12\\Dataset_TA_pkl\\LATENT_32640_2CHANNEL\\'
path_file = pkl_dir + 'path_and_label.csv'
train_dir = pkl_dir + 'train_set.csv'
valid_dir = pkl_dir + 'valid_set.csv'
test_dir = pkl_dir + 'test_set.csv'

In [ ]:
datasetsTrain = PickleLatentDatasetLoader(
    annotations_file=train_dir,
    dir=pkl_dir
)
datasetsValid = PickleLatentDatasetLoader(
    annotations_file=valid_dir,
    dir=pkl_dir
)
datasetsTest = PickleLatentDatasetLoader(
    annotations_file=test_dir,
    dir=pkl_dir
)

In [ ]:
print(len(datasetsTest))

In [ ]:
# data loader
train_dataloader = DataLoader(datasetsTrain, batch_size=50)
validation_dataloader = DataLoader(datasetsValid, batch_size=50)
test_dataloader = DataLoader(datasetsTest, batch_size=50)

In [ ]:
conv_layer = torch.nn.Sequential(
    torch.nn.Conv2d(64, 32, 3, stride=1, padding=0),
    torch.nn.ReLU(True),
    torch.nn.Conv2d(32, 16, 3, stride=1, padding=0),
    torch.nn.ReLU(True),
    torch.nn.Conv2d(16, 16, 3, stride=1, padding=0),
    torch.nn.ReLU(True),
    torch.nn.Conv2d(16, 8, 3, stride=1, padding=0),
)

In [ ]:
#MODEL 1 (Flatten and FC only)
class LitClassifier(pl.LightningModule):
    def __init__(self, input_shape, learning_rate=1e-4):
        super().__init__()

        # log hyperparameters
        self.save_hyperparameters()
        self.learning_rate = learning_rate
        self.loss_fn = torch.nn.BCEWithLogitsLoss()

        # variable
        self.predictions = np.array([])
        self.targets = np.array([])
        self.predictions_val = np.array([])
        self.targets_val = np.array([])
        self.predictions_test = np.array([])
        self.targets_test = np.array([])
        self.f1_fn = BinaryF1Score()
        self.acc_fn = BinaryAccuracy()
        self.f1_train = 0
        self.f1_val = 0
        self.f1_test = 0
        self.acc_train = 0
        self.acc_val = 0
        self.acc_test = 0

        # Convolutional
        self.conv_layer = torch.nn.Sequential(
            torch.nn.Conv2d(64, 32, 3, stride=1, padding=0),
            torch.nn.ReLU(True),
            torch.nn.Conv2d(32, 16, 3, stride=1, padding=0),
            torch.nn.ReLU(True),
            torch.nn.Conv2d(16, 16, 3, stride=1, padding=0),
            torch.nn.ReLU(True),
            torch.nn.Conv2d(16, 8, 3, stride=1, padding=0),
        )

        # Flatten
        self.flatten_layer = torch.nn.Flatten(start_dim=1)

        # Fully Connected
        n_sizes = self._get_output_shape(input_shape)
        self.fc_layer = torch.nn.Sequential(
            torch.nn.Linear(n_sizes, 512),
            torch.nn.ReLU(True),
            torch.nn.Linear(512, 128),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(128, 1)
        )

    def _get_output_shape(self, shape):
        batch_size = 50
        input = torch.autograd.Variable(torch.rand(batch_size, *shape))
        output_feat = self.conv_layer(input)
        n_size = output_feat.data.view(batch_size, -1).size(1)
        return n_size*2

    def forward(self, x1, x2):
         x1 = self.conv_layer(x1)
         x2 = self.conv_layer(x2)
         x_combined = torch.stack([x1,x2], dim=0)
         x_flat = self.flatten_layer(x_combined)
         y_logits = self.fc_layer(x_flat)
         y_pred = torch.round(torch.sigmoid(y_logits))
         return y_pred

    def training_step(self, batch, batch_idx):
        x1, x2, y = batch
        x1 = self.conv_layer(x1)
        x2 = self.conv_layer(x2)
        x_combined = torch.stack([x1,x2], dim=1)
        x_flat = self.flatten_layer(x_combined)
        y_logits = self.fc_layer(x_flat)
        y_logits = y_logits.squeeze()
        loss = self.loss_fn(y_logits, y)
        acc = self.acc_fn(y_logits, y)
        self.log("train_loss", loss)
        self.log("train_acc", acc)
        y_preds = torch.round(torch.sigmoid(y_logits.cpu())).detach().numpy()
        self.predictions = np.append(self.predictions, y_preds, axis=0)
        self.targets = np.append(self.targets, y.cpu(), axis=0)
        return {"loss": loss, "acc": acc}

    def validation_step(self, val_batch, batch_idx):
        x1, x2, y = val_batch
        x1 = self.conv_layer(x1)
        x2 = self.conv_layer(x2)
        x_combined = torch.stack([x1,x2], dim=1)
        x_flat = self.flatten_layer(x_combined)
        y_logits = self.fc_layer(x_flat)
        y_logits = y_logits.squeeze()
        val_loss = self.loss_fn(y_logits, y)
        val_acc = self.acc_fn(y_logits, y)
        self.log("val_loss", val_loss)
        self.log("val_acc", val_acc)
        y_preds = torch.round(torch.sigmoid(y_logits.cpu())).detach().numpy()
        self.predictions_val = np.append(self.predictions_val, y_preds, axis=0)
        self.targets_val = np.append(self.targets_val, y.cpu(), axis=0)
        return {"val_loss": val_loss, "val_acc": val_acc}

    def test_step(self, test_batch, batch_idx):
        x1, x2, y = test_batch
        x1 = self.conv_layer(x1)
        x2 = self.conv_layer(x2)
        x_combined = torch.stack([x1,x2], dim=1)
        x_flat = self.flatten_layer(x_combined)
        y_logits = self.fc_layer(x_flat)
        y_logits = y_logits.squeeze()
        test_loss = self.loss_fn(y_logits, y)
        test_acc = self.acc_fn(y_logits, y)
        self.log("test_loss", test_loss)
        self.log("test_acc", test_acc)
        y_preds = torch.round(torch.sigmoid(y_logits.cpu())).detach().numpy()
        conf_matrix = metrics.confusion_matrix(y.cpu(), y_preds)
        conf_matrix = np.flip(conf_matrix).T
        legend = ['Lie','Truth']
        legend2 = [['(TP)','(FP)'],['(FN)','(TN)']]
        fig, ax = plt.subplots(figsize=(5, 5))
        ax.matshow(conf_matrix, cmap=plt.cm.Blues, alpha=0.3)
        ax.set_xticklabels([''] + legend)
        ax.set_yticklabels([''] + legend)
        for i in range(conf_matrix.shape[0]):
            for j in range(conf_matrix.shape[1]):
                ax.text(x=j, y=i, s=(str(conf_matrix[i, j]) + ' ' + legend2[i][j]), va='center', ha='center', size='xx-large')

        plt.ylabel('Predictions', fontsize=20)
        plt.title('Actual', fontsize=20)
        plt.xticks(fontsize=18)
        plt.yticks(fontsize=18)
        plt.tight_layout(pad=1)
        y_preds = torch.round(torch.sigmoid(y_logits))
        f1score = self.f1_fn(y_preds, y)
        print('F1-score:', f1score)
        plt.show()
        return {"test_loss": test_loss, "test_acc": test_acc}

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=self.learning_rate)
        return optimizer

In [ ]:
classifier = LitClassifier(input_shape=(64,10,51))

In [ ]:
class LoggingCallback(Callback):
    def __init__(self):
        super().__init__()
        self.f1 = BinaryF1Score()

    def on_train_epoch_start(self, trainer: "pl.Trainer", pl_module: "pl.LightningModule") -> None:
        pl_module.predictions = np.array([])
        pl_module.targets = np.array([])

    def on_train_epoch_end(self, trainer: "pl.Trainer", pl_module: "pl.LightningModule") -> None:
        pl_module.f1_train = pl_module.f1_fn(torch.tensor(pl_module.predictions.squeeze()), torch.tensor(pl_module.targets.squeeze()))
        pl_module.log('F1_Score', pl_module.f1_train)

    def on_validation_epoch_start(self, trainer: "pl.Trainer", pl_module: "pl.LightningModule") -> None:
        pl_module.predictions_val = np.array([])
        pl_module.targets_val = np.array([])

    def on_validation_epoch_end(self, trainer: "pl.Trainer", pl_module: "pl.LightningModule") -> None:
        pl_module.f1_val = pl_module.f1_fn(torch.tensor(pl_module.predictions_val.squeeze()), torch.tensor(pl_module.targets_val.squeeze()))
        pl_module.log('F1_Score_val', pl_module.f1_val)

In [ ]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    monitor="F1_Score_val",
    mode="max",
    # dirpath="E:\\Nicko\\TUGAS_AKHIR\\ClasifierConvFc\\runs\\run_1",
    dirpath="E:\\Nicko\\TUGAS_AKHIR\\ClasifierFc\\runs\\run_1",
    filename="classifier-{epoch:02d}-{F1_Score_val:.2f}",
    save_top_k=5,
    save_last=True
)
logging_callback = LoggingCallback()

In [ ]:
# wandb_logger = WandbLogger(project='classifier_conv_fc', name='run_1', save_dir='E:\\Nicko\\TUGAS_AKHIR\\ClasifierConvFc\\runs')
wandb_logger = WandbLogger(project='classifier_fc', name='run_1', save_dir='E:\\Nicko\\TUGAS_AKHIR\\ClasifierFc\\runs')

In [ ]:
trainer = pl.Trainer(max_epochs=5000, devices=1, accelerator='gpu', log_every_n_steps=3, logger=wandb_logger, callbacks=[checkpoint_callback, logging_callback])
trainer.fit(classifier, train_dataloader, validation_dataloader)
wandb.finish()

In [ ]:
classifier_load = LitClassifier(input_shape=(64,10,51)).load_from_checkpoint(checkpoint_path='E:\\Nicko\\TUGAS_AKHIR\\ClasifierConvFc\\runs\\run_1\\classifier-epoch=02-F1_Score_val=0.56.ckpt')

In [ ]:
trainer = pl.Trainer(max_epochs=5000, devices=1, accelerator='gpu', log_every_n_steps=3, logger=False)
trainer.test(classifier_load, test_dataloader)